# Pre-process WRF DATM

In [1]:
import os

import xarray as xr
import functools
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import math
import glob
import re

%matplotlib inline

In [2]:
import warnings
warnings.simplefilter('ignore')

### User-defined parameters

In [3]:
#1.
#path to regional wrf data that you want to subset from
#wrf_data = '/glade/scratch/adamhb/met_data/wrf_9km_1982_2020/'

# Reanalysis
#wrf_data = '/glade/work/xiugao/fates-input/ca-wrf-grassland/CLM1PT_data'

# Projections
wrf_data ='/glade/work/xiugao/fates-input/ca-wrf-grassland/ec-earth3-ssp370-2015-2099'

#2.
#output path where you want the subsetted, pre-processed data to go
output_path = '/glade/work/adamhb/input_data/CZ2_wrf_ssp370/CLM1PT_data/'

#3.
#Coordinates of site where you want to subset data

#CZ2: 37.0311,-119.256599
#STAN: 38.177648,-119.978692

target_wrf_lat = 37.0311
target_wrf_lon = -119.256599

### User should not need to change anything after this point

### Constants

In [4]:
# Molar mass of water vapour [kg/mol]
mm_h2o  = 0.01801505
# Molar mass of dry air [kg/mol]
mm_dry  = 0.02897
# Molar mass ratio
eps_mol = mm_h2o / mm_dry
# Saturation vapour pressure at 273.15K [Pa]
esat_0C = 611.65685464
frac_2_pc = 100
degC_2_K  = 273.15

site_refhgt=2

# Convert the user-defined target lon to be non-negative in the western hemisphere.
# This is to match clm format
target_lat = target_wrf_lat

if target_wrf_lon < 0:
    target_lon = target_wrf_lon + 360

### Get WRF LAT and LON indices at the closest WRF point to the target site

In [5]:
def dist_func(a,b):
    return math.sqrt(a**2 + b**2)

def getDiff(d,var,target,r,c):
    return abs(d[var][r,c].values - target)

def getClosestWRFPoint(xds,lat_target,lon_target):

    output = {'lat':[],'lon':[],'dist':[]}
    for r in range(xds.LATIXY.shape[0]):
        for c in range(xds.LATIXY.shape[1]):

            lon_diff = getDiff(xds,'LONGXY',lon_target,r,c)
            lat_diff = getDiff(xds,'LATIXY',lat_target,r,c)
            dist = dist_func(lon_diff, lat_diff)
            output['lat'].append(r)
            output['lon'].append(c)
            output['dist'].append(dist)
            
    output = pd.DataFrame(output)
    closest = output.loc[output.dist == output.dist.min()]
    lat_index = closest.lat.values
    lon_index = closest.lon.values
    return lat_index[0], lon_index[0]

def get_last_file_of_sim(sim_path):
    files = sorted(os.listdir(sim_path))
    full_files = [os.path.join(sim_path,f) for f in files]
    last_file = full_files[-1]
    print("last file:",last_file)
    return last_file

def return_sorted_wrf_files(directory_path):
    matching_files = []
    pattern = re.compile(r'\d{4}-\d{2}\.nc')

    for filename in os.listdir(directory_path):
        if pattern.match(filename):
            matching_files.append(os.path.join(directory_path, filename))

    return sorted(matching_files)


#Get lat and lon indices of WRF data at closest point to target location (i.e. the site)
ds_01 = xr.open_dataset(return_sorted_wrf_files(wrf_data)[-1],decode_times = False)
wrf_lat_index, wrf_lon_index = getClosestWRFPoint(ds_01,target_wrf_lat,target_wrf_lon)


print("Target Latitude (i.e. site latitude):",target_wrf_lat)
print("Latitude index of closest WRF point:",wrf_lat_index)
print("Latitude of closest WRF point:",ds_01.LATIXY.sel(lat = wrf_lat_index,lon = wrf_lon_index).values)

print("\n")

print("Target Longitude (i.e. site longitude):", target_wrf_lon)
print("Longitude index of closest WRF point:",wrf_lon_index)
print("Latitude of closest WRF point:", ds_01.LONGXY.sel(lat = wrf_lat_index,lon = wrf_lon_index).values)

Target Latitude (i.e. site latitude): 37.0311
Latitude index of closest WRF point: 74
Latitude of closest WRF point: 37.07170104980469


Target Longitude (i.e. site longitude): -119.256599
Longitude index of closest WRF point: 74
Latitude of closest WRF point: -119.22645568847656


### Functions to pre-process WRF data

In [6]:
def cast_all_vars_to_float32(xds):
    vars = list(xds.keys())
    for v in vars:
        float_32_var = xds[v].astype("float32")
        xds[v] = float_32_var
    return xds

def change_LATIXY_and_LONGXY(xds,lat,lon):
    LONGXY_attrs = xds.LONGXY.attrs
    LATIXY_attrs = xds.LATIXY.attrs
    new_LATIXY = xr.DataArray(data = np.array(lat,dtype = "float32").reshape(1,1),
                              dims = ['lat','lon'],
                              attrs = LATIXY_attrs)
    new_LONGXY = xr.DataArray(data = np.array(lon,dtype = "float32").reshape(1,1),
                              dims = ['lat','lon'],
                              attrs = LONGXY_attrs)
    
    xds['LATIXY'] = new_LATIXY
    xds['LONGXY'] = new_LONGXY
    return xds

def getRH_from_QBOT(QBOT,PSRF,TBOT):

    EBOT = PSRF * QBOT / (eps_mol + (1. - eps_mol) * QBOT)
    ESAT = esat_0C * math.exp( 17.67 * (TBOT - degC_2_K) / (TBOT - 29.65))
    RH   = frac_2_pc * EBOT / ESAT
    return RH

    
def put_RH_in_array(xds):
    QBOT_array = list(xds.QBOT.values)
    PSRF_array = list(xds.PSRF.values)
    TBOT_array = list(xds.TBOT.values)
    times = xds.time.values

    RH_array = np.zeros(len(QBOT_array))
    for i in range(len(QBOT_array)):
        RH_array[i] = getRH_from_QBOT(QBOT_array[i],PSRF_array[i],TBOT_array[i])

    return RH_array


def add_RH_to_data(xds):
    times = xds.time.values
    n = len(times)
    RH_array = put_RH_in_array(xds)

    RH_xda = xr.DataArray(
                 data = np.array(RH_array).reshape(n,1,1),
                 dims = ['time','lat','lon'],
                 coords = {"time": times},
                 attrs = {
                     'longname': 'relative humidity',
                     'units' : '%',
                     'mode' : 'time-dependent'
                     })
    xds['RH'] = RH_xda

def add_ZBOT_to_data(xds):
    times = xds.time.values
    n = len(xds.time.values)                    
    ZBOT_array = np.array([site_refhgt] * n).reshape(n,1,1) 
                        
    ZBOT_xda = xr.DataArray(
                 data = ZBOT_array.reshape(n,1,1),
                 dims = ['time','lat','lon'],
                 coords = {"time": times},
                 attrs = {
                     'longname': 'observation_height',
                     'units' : 'm',
                     'mode' : 'time-dependent'
                     })
    xds['ZBOT'] = ZBOT_xda

    
def add_EDGE_vars(xds):
    
    add = [0.5,-0.5,0.5,-0.5]
    var = ['LATIXY','LATIXY','LONGXY','LONGXY']
    for i,d in enumerate(['N','S','E','W']):
        edge_var_name = "EDGE" + d
        edge_value = xds[var[i]].values + add[i]
        edge_xar = xr.DataArray(
                     data = edge_value[0,:],
                     dims = ['scalar'],
                     attrs = {
                     'longname': 'edge of datm',
                     'units' : 'degrees',
                     'mode' : 'time-dependent'
                     })
        xds[edge_var_name] = edge_xar

            
def subset_single_file(file):
    ds = xr.open_dataset(file, decode_times = False)
    
    #subset
    point_data = ds.sel(lat = slice(wrf_lat_index,(wrf_lat_index+1)),
                        lon = slice(wrf_lon_index,(wrf_lon_index+1)))
    
    #get time attributes
    time_attrs = point_data.time.attrs
    
    #add variables
    add_ZBOT_to_data(point_data)
    add_RH_to_data(point_data)
        
    #reset lat and lon to be same as surf data
    point_data = change_LATIXY_and_LONGXY(cast_all_vars_to_float32(point_data),target_lat,target_lon)
    
    #add edge values
    add_EDGE_vars(point_data)
    
    #re-assign time attributes to data
    point_data.time.attrs = time_attrs
    
    #write output
    new_file_name = output_path + "/" + os.path.basename(file)
    point_data.to_netcdf(new_file_name, format = "NETCDF3_64BIT", mode = "w")
    ds.close()
    point_data.close()
    print("Finished",new_file_name)

### Apply pre-processing functions to WRF domain files

In [7]:
#files = glob.glob(wrf_data + "/*1980*.nc")
files = return_sorted_wrf_files(wrf_data)

In [8]:
for f in sorted(files):
    subset_single_file(f)

Finished /glade/work/adamhb/input_data/CZ2_wrf_ssp370/CLM1PT_data//2014-09.nc
Finished /glade/work/adamhb/input_data/CZ2_wrf_ssp370/CLM1PT_data//2014-10.nc
Finished /glade/work/adamhb/input_data/CZ2_wrf_ssp370/CLM1PT_data//2014-11.nc
Finished /glade/work/adamhb/input_data/CZ2_wrf_ssp370/CLM1PT_data//2014-12.nc
Finished /glade/work/adamhb/input_data/CZ2_wrf_ssp370/CLM1PT_data//2015-01.nc
Finished /glade/work/adamhb/input_data/CZ2_wrf_ssp370/CLM1PT_data//2015-02.nc
Finished /glade/work/adamhb/input_data/CZ2_wrf_ssp370/CLM1PT_data//2015-03.nc
Finished /glade/work/adamhb/input_data/CZ2_wrf_ssp370/CLM1PT_data//2015-04.nc
Finished /glade/work/adamhb/input_data/CZ2_wrf_ssp370/CLM1PT_data//2015-05.nc
Finished /glade/work/adamhb/input_data/CZ2_wrf_ssp370/CLM1PT_data//2015-06.nc
Finished /glade/work/adamhb/input_data/CZ2_wrf_ssp370/CLM1PT_data//2015-07.nc
Finished /glade/work/adamhb/input_data/CZ2_wrf_ssp370/CLM1PT_data//2015-08.nc
Finished /glade/work/adamhb/input_data/CZ2_wrf_ssp370/CLM1PT_dat